In [1]:
import pandas as pd
import numpy as np

In [2]:
aluminum_division = 12
near_k = 2
weight_limit = 56

In [3]:
df = pd.read_csv(r"C:\Users\tzuli\Documents\python\teco\data\train\train_data.csv")
df = df.query("(初始_L側不平衡量 >= 4) or (初始_F側不平衡量 >= 4)").reset_index(drop = True)
print(df.shape)
df.head(3)

(120, 16)


,日期,工號,序號,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,最終_L側角度,最終_L側不平衡量,最終_F側角度,最終_F側不平衡量
0,2023-06-07,FC236001,1,1000,277.0,12.5,145,24.9,277.0,12.0,145,24.0,312,1.4,56,2.2
1,2023-06-07,FC234613,1,1000,298.0,155.0,267,64.3,298.0,150.0,267,60.0,9,23.0,197,3.6
2,2023-06-07,FC234613,2,1000,9.0,23.0,197,3.6,9.0,20.0,197,3.0,106,3.7,178,2.7


### 填補加總

In [4]:
df1 = df.copy()
achieve_l = df1.query("(初始_L側不平衡量 < 4)").index
df1.loc[achieve_l, ['平衡_L側角度', '平衡_L側配重']] = 0
achieve_f = df1.query("(初始_F側不平衡量 < 4)").index
df1.loc[achieve_f, ['初始_F側角度', '初始_F側不平衡量']] = 0

In [5]:
df_total = df1.groupby("工號").first()
df_total1 = df1.groupby("工號").agg({"平衡_L側配重": sum, "平衡_F側配重": sum})
df_total1.columns = ["平衡_L側加總", "平衡_F側加總"]
df_total = pd.merge(df_total, df_total1, left_index = True, right_index = True)
df_total["L側重量差"] = df_total["平衡_L側加總"] - df_total["初始_L側不平衡量"]
df_total["F側重量差"] = df_total["平衡_F側加總"] - df_total["初始_F側不平衡量"]
df_total = df_total.reset_index()

count_l = df1.query("平衡_L側配重 > 0").groupby("工號")["序號"].count().to_dict()
df_total["L側次數"] = df_total["工號"].map(count_l)
count_f = df1.query("平衡_F側配重 > 0").groupby("工號")["序號"].count().to_dict()
df_total["F側次數"] = df_total["工號"].map(count_f)

df_total["L側根數"] = ((df_total["初始_L側不平衡量"] // 60) + ((df_total["初始_L側不平衡量"] % 60) > 0).astype(int)).astype(int)
df_total["F側根數"] = ((df_total["初始_F側不平衡量"] // 60) + ((df_total["初始_F側不平衡量"] % 60) > 0).astype(int)).astype(int)

df_total.head(3)

,工號,日期,序號,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,...,最終_F側角度,最終_F側不平衡量,平衡_L側加總,平衡_F側加總,L側重量差,F側重量差,L側次數,F側次數,L側根數,F側根數
0,FC225607-1,2023-06-10,1,590,266.0,114.0,323,122.0,266.0,120.0,...,279,22.0,132.0,137.0,18.0,15.0,2,2,2,3
1,FC225607-2,2023-06-10,1,590,298.0,317.0,11,113.0,298.0,300.0,...,24,19.1,416.0,122.0,99.0,9.0,4,3,6,2
2,FC225607-3,2023-06-10,1,590,298.0,130.0,65,174.0,298.0,120.0,...,68,37.0,148.0,202.0,18.0,28.0,3,3,3,3


In [6]:
from prog.tools import calculate_angle_proportion
aluminum_division = 12
angle_init = np.linspace(0, 360, (aluminum_division + 1)).astype(int)
print(f"angle_init = {angle_init}")

df_l = df_total[["初始_L側角度", "初始_L側不平衡量"]]
df_f = df_total[["初始_F側角度", "初始_F側不平衡量"]]

df_l = calculate_angle_proportion(df_l, angle_init, aluminum_division, side = "L")            
df_f = calculate_angle_proportion(df_f, angle_init, aluminum_division, side = "F")

df_total["l_target"] = df_l["angle_target"].astype(int)
df_total["f_target"] = df_f["angle_target"].astype(int)

df_total = df_total.iloc[:, [0, 20, 24, 4, 22, 5, 16, 18, 21, 25, 6, 23, 7, 17, 19]]
df_total = df_total.sort_values(["L側次數", "F側次數", "L側根數", "F側根數"], ascending = False).reset_index(drop = True)
df_total

df_total.head(3)

angle_init = [  0  30  60  90 120 150 180 210 240 270 300 330 360]


,工號,L側次數,l_target,初始_L側角度,L側根數,初始_L側不平衡量,平衡_L側加總,L側重量差,F側次數,f_target,初始_F側角度,F側根數,初始_F側不平衡量,平衡_F側加總,F側重量差
0,FC237002-2,4,37,266.0,6,317.0,439.0,122.0,3,37,214,4,184.0,219.0,35.0
1,FC225607-2,4,37,298.0,6,317.0,416.0,99.0,3,55,11,2,113.0,122.0,9.0
2,FC225607-3,3,37,298.0,3,130.0,148.0,18.0,3,37,65,3,174.0,202.0,28.0


### 數據統計

In [7]:
# 小於60的次數分布
l_count = df_total.query("初始_L側不平衡量 <= 60")["L側次數"].value_counts().sort_index().to_frame().reset_index()
f_count = df_total.query("初始_F側不平衡量 <= 60")["F側次數"].value_counts().sort_index().to_frame().reset_index()
count_less60 = pd.concat([l_count, f_count], axis = 1)
count_less60

,L側次數,count,F側次數,count
0,1,10,1,8
1,2,9,2,24


In [8]:
# 大於60的次數分布
l_count = df_total.query("初始_L側不平衡量 > 60")["L側次數"].value_counts().sort_index().to_frame().reset_index()
f_count = df_total.query("初始_F側不平衡量 > 60")["F側次數"].value_counts().sort_index().to_frame().reset_index()
count_more60 = pd.concat([l_count, f_count], axis = 1)
count_more60

,L側次數,count,F側次數,count
0,1,4,1.0,1.0
1,2,27,2.0,19.0
2,3,4,3.0,4.0
3,4,2,NaN,NaN


In [9]:
# 小於60的角度分類
l_count = df_total.query("初始_L側不平衡量 <= 60")["l_target"].value_counts().sort_index().to_frame().reset_index()
f_count = df_total.query("初始_F側不平衡量 <= 60")["f_target"].value_counts().sort_index().to_frame().reset_index()
angle_less60 = pd.concat([l_count, f_count], axis = 1)
angle_less60

,l_target,count,f_target,count
0,10,1,37.0,17.0
1,37,11,55.0,15.0
2,55,7,NaN,NaN


In [10]:
# 大於60的角度分類
l_count = df_total.query("初始_L側不平衡量 > 60")["l_target"].value_counts().sort_index().to_frame().reset_index()
f_count = df_total.query("初始_F側不平衡量 > 60")["f_target"].value_counts().sort_index().to_frame().reset_index()
angle_more60 = pd.concat([l_count, f_count], axis = 1)
angle_more60

,l_target,count,f_target,count
0,10,1,10,1
1,37,20,37,16
2,55,16,55,7


In [11]:
# 填補次數超過三次的角度分類
l_count = df_total.query("L側次數 >= 3")["l_target"].value_counts().sort_index().to_frame().reset_index()
f_count = df_total.query("F側次數 >= 3")["f_target"].value_counts().sort_index().to_frame().reset_index()
angle_more60= pd.concat([l_count, f_count], axis = 1)
angle_more60

,l_target,count,f_target,count
0,37.0,6.0,37,3
1,NaN,NaN,55,1


In [12]:
diff_median = df_total[["L側重量差", "F側重量差"]].median().to_frame().T
l_diff_median = df_total.query("(初始_L側不平衡量 > @weight_limit)")["L側重量差"].median()
f_diff_median = df_total.query("(初始_F側不平衡量 > @weight_limit)")["F側重量差"].median()
diff_median.loc[1] = [l_diff_median, f_diff_median]
diff_median.index = ["all", f"初始 > {weight_limit}"]
diff_median

,L側重量差,F側重量差
all,5.0,2.85
初始 > 56,6.4,8.00


### 各角度填補數值

In [13]:
angle_init = np.linspace(0, 360, (aluminum_division + 1)).astype(int)
angle_col = [f"L_{i}" for i in angle_init[:-1]] + [f"F_{i}" for i in angle_init[:-1]]
df[angle_col] = None
df.head(3)

,日期,工號,序號,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,...,F_60,F_90,F_120,F_150,F_180,F_210,F_240,F_270,F_300,F_330
0,2023-06-07,FC236001,1,1000,277.0,12.5,145,24.9,277.0,12.0,...,None,None,None,None,None,None,None,None,None,None
1,2023-06-07,FC234613,1,1000,298.0,155.0,267,64.3,298.0,150.0,...,None,None,None,None,None,None,None,None,None,None
2,2023-06-07,FC234613,2,1000,9.0,23.0,197,3.6,9.0,20.0,...,None,None,None,None,None,None,None,None,None,None


In [14]:
for i in range(len(df)):
    angle = df.loc[i, "初始_L側角度"]
    weight = df.loc[i, "平衡_L側配重"]
    closest_index = np.argsort(np.abs(angle_init - angle))[:near_k] # 與angle_ori最接近的K個角度
    near_angle = angle_init[closest_index]
    gap = (angle - near_angle[0]) / (near_angle[1] - near_angle[0])

    if near_angle[0] == 360:
        near_angle[0] = 0
    if near_angle[1] == 360:
        near_angle[1] = 0

    if gap < 0.3:
        df.loc[i, f"L_{near_angle[0]}"] = weight
    elif gap > 0.7:        
        df.loc[i, f"L_{near_angle[1]}"] = weight
    else:
        df.loc[i, f"L_{near_angle[0]}"] = weight/2
        df.loc[i, f"L_{near_angle[1]}"] = weight/2

In [15]:
for i in range(len(df)):
    angle = df.loc[i, "初始_F側角度"]
    weight = df.loc[i, "平衡_F側配重"]
    closest_index = np.argsort(np.abs(angle_init - angle))[:near_k] # 與angle_ori最接近的K個角度
    near_angle = angle_init[closest_index]
    gap = (angle - near_angle[0]) / (near_angle[1] - near_angle[0])

    if near_angle[0] == 360:
        near_angle[0] = 0
    if near_angle[1] == 360:
        near_angle[1] = 0

    if gap < 0.3:
        df.loc[i, f"F_{near_angle[0]}"] = weight
    elif gap > 0.7:        
        df.loc[i, f"F_{near_angle[1]}"] = weight
    else:
        df.loc[i, f"F_{near_angle[0]}"] = weight/2
        df.loc[i, f"F_{near_angle[1]}"] = weight/2

In [16]:
for i in range(len(df)):
    if df.loc[i, "序號"] != 1:
        df.loc[i, "工號"] = None
        df.loc[i, "日期"] = None
df.head()

,日期,工號,序號,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,...,F_60,F_90,F_120,F_150,F_180,F_210,F_240,F_270,F_300,F_330
0,2023-06-07,FC236001,1,1000,277.0,12.5,145,24.9,277.0,12.0,...,None,None,None,24.0,None,None,None,None,None,None
1,2023-06-07,FC234613,1,1000,298.0,155.0,267,64.3,298.0,150.0,...,None,None,None,None,None,None,None,60.0,None,None
2,None,None,2,1000,9.0,23.0,197,3.6,9.0,20.0,...,None,None,None,None,1.5,1.5,None,None,None,None
3,2023-06-07,FC236531,1,1000,15.0,111.0,165,30.2,15.0,90.0,...,None,None,None,15.0,15.0,None,None,None,None,None
4,None,None,2,1000,3.0,24.7,140,4.7,3.0,24.0,...,None,None,2.0,2.0,None,None,None,None,None,None


writer = pd.ExcelWriter(r"C:\Users\tzuli\Documents\python\teco\data\train\eda.xlsx", engine = 'xlsxwriter')
df.to_excel(writer, sheet_name = '各角度填補', index = False)
df_total.to_excel(writer, sheet_name = '填補加總', index = False)
writer.close()